# MYERS VS HISTOGRAM

In [ ]:
import os, sys, csv
from os import path
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
analyze_dir = userhome + r'/different-diff/analyze/collect_metrics/analyze_crate.io/'
project = 'Crate.IO'

# Open diff dataset for Myers and Histogram

In [ ]:
df_diffmyers = pd.read_csv(analyze_dir + '02_diff-file/diffmyers-metrics.csv')
df_diffhist = pd.read_csv(analyze_dir + '02_diff-file/diffhist-metrics.csv')

In [ ]:
df_diffmyers

In [ ]:
df_diffhist

# Merge two datasets

In [ ]:
df_merge = df_diffmyers.merge(df_diffhist, on=['commit_id','parent_id','filename'], 
                                            how='outer', suffixes=('_myers', '_histogram'))
df_merge.fillna(0, inplace=True)

In [ ]:
fields = ['commit_id','parent_id','filename','#insertions_myers','#insertions_histogram','#deletions_myers','#deletions_histogram']
df_merge = df_merge[fields]
df_merge

# Capturing only filename with different number of added and deleted lines

In [ ]:
df_different = []
df_same = []
for row in range(0, len(df_merge)):
    ins_myers = df_merge.iloc[row][3]
    ins_hist = df_merge.iloc[row][4]
    del_myers = df_merge.iloc[row][5]
    del_hist = df_merge.iloc[row][6]
    if (ins_myers != ins_hist) or (del_myers != del_hist):
        df_different.append(df_merge.iloc[row])
    else:
        df_same.append(df_merge.iloc[row])

print ("Process is complete")
print ("------------------------------------------------------")
print ("Project name : {}".format(project))
print ("The number of commits that have different result: %i" %(len(df_different)))
print ("The number of commits that have same result: %i" %(len(df_same)))

In [ ]:
with open(analyze_dir + '06_myers_vs_histogram/number_of_commits_with_different_result.csv','w') as diffcsv:
    writers = csv.writer(diffcsv, delimiter=',')
    writers.writerow(fields)
    for item in df_different:
        writers.writerow(item)
print ("The CSV file that has different result has been created")

In [ ]:
with open(analyze_dir + '06_myers_vs_histogram/number_of_commits_with_same_result.csv','w') as diffcsv:
    writers = csv.writer(diffcsv, delimiter=',')
    writers.writerow(fields)
    for item in df_same:
        writers.writerow(item)
print ("The CSV file that has same result has been created")

In [ ]:
dframe = pd.read_csv(analyze_dir + '06_myers_vs_histogram/number_of_commits_with_different_result.csv')
dframe

In [ ]:
percent_of_diffresult = (len(df_different)/len(df_merge))*100
print ("The percentage of commits that have different results in " + project + " Project = {0:.2f}%".
       format(percent_of_diffresult))